In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.keys import Keys
import pyautogui
import pyperclip

In [ ]:
URL = 'https://comic.naver.com/webtoon'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')

week = ['tab=mon', 'tab=tue', 'tab=wed', 'tab=thu', 'tab=fri', 'tab=sat', 'tab=sun', 'tab=finish']
ko_week = ['월', '화', '수', '목', '금', '토', '일', '완결']

In [ ]:
id_list = [] ; title_list = [] ; author_list = [] ; day_list = [] ; genre_list = [] ; 
story_list = [] ; platform_list = [] ; img_list = [] ; tag_list = [] ; 
grade_list = [] ; viewer_list = [] ; series_list = [] ; regist_list = [] ; href_lists = []
num = 0

In [ ]:
driver = webdriver.Chrome() # 크롬 사용하니까, 크롬 드라이버를 현재 파이썬 코드가 있는 폴더로!!

def naver_login():
    login = 'https://nid.naver.com/nidlogin.login?url=https%3A%2F%2Fcomic.naver.com%2Fwebtoon%3Ftab%3Dmon'
    driver.implicitly_wait(2)
    driver.maximize_window()
    driver.get(login)
    
#     driver.find_element_by_css_selector("#id_email_2_label").click()
    driver.find_element_by_xpath('//*[@id="id"]').click()
    pyperclip.copy('{아이디}')
    pyautogui.hotkey('ctrl','v')
    sleep(5)
    
#     driver.find_element_by_css_selector('#id_password_3_label').click()
    driver.find_element_by_xpath('//*[@id="pw"]').click()
    pyperclip.copy('{비번}')
    pyautogui.hotkey('ctrl','v')
    sleep(5)

    #로그인 버튼 
#     login_send = browser.find_element_by_css_selector("#login-form > fieldset > div.wrap_btn > button.btn_g.btn_confirm.submit")
    login_send = driver.find_element_by_xpath('//*[@id="log.login"]')
    login_send.click()
#     driver.back()

In [ ]:
driver.get(URL)
driver.implicitly_wait(3) #로딩이 끝날 때까지 10초까지는 기다려줌
naver_login() #로그인 

In [ ]:
for i in range(7):
    sleep(0.5)
    uri = URL + "?" + week[i]
    
    driver.get(uri) #요일별로 링크 가져옴

    html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
    soup = BeautifulSoup(html, 'html.parser')
    
    sleep(0.5)

    #스크롤 전 높이
    before_h = driver.execute_script("return window.scrollY")

    #무한 스크롤
    while True:
        #맨 아래로 스크롤을 내린다
        driver.find_element_by_css_selector("body").send_keys(Keys.END)

        #스크롤 사이 페이지 로딩 시간
        sleep(1)

        #스크롤 후 높이
        after_h = driver.execute_script("return window.scrollY")

        if after_h == before_h:
            break

        before_h = after_h
        
    ## 웹툰 디테일 링크 + 별점

    href_list = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/ul/li/a')
    grades = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/ul/li/div/div/span')
#     authors_li = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/ul/li/div/div/a')
    
    page_list = []
    for page in href_list:
        page_list.append(page.get_attribute('href'))

    scores = []
    for grade in grades:
        scores.append(grade.text.replace("별점", "").replace("\n", ""))

#     authors = []
#     for author in authors_li:
#         authors.append(author.text)
        
#     print(len(authors))
#     for i in range(len(authors))
#         print(authors[i])
    
    idx = 0

    for href in page_list :
#     href = href_list[i].get_attribute('href')
#     print(href_list[i].get_attribute('href'))
#     print(grades[i].text.replace("별점", "").replace("\n", ""))
#     print(authors[i].text)

        try:
            author = driver.find_element_by_xpath('//*[@id="content"]/div[1]/ul/li[' + str(idx+1) + ']/div/div[1]/a')
        except:
            author = driver.find_element_by_xpath('//*[@id="content"]/div[1]/ul/li['+ str(idx+1) + ']/div/a[2]')
        print(author.text)
        author_info = author.text
        
        sleep(2)

        driver.get(href + "&page=1&sort=ASC") # 디테일 이동
    
        html = requests.get(href).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
        soup = BeautifulSoup(html, 'html.parser')
    
        sleep(2)
    
        day = ko_week[i]
    
        ## 제목
        title = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/h2').text
        if (title in title_list):  # 요일 두 개 이상이면 요일만 추가함
                    day_list[title_list.index(title)] += ', ' + day
                    driver.back()
                    continue
                
#         print(title)
        title_list.append(title)
    
        grade_list.append(scores[idx])
    
        id_list.append(num) ; num += 1  # id 리스트에 추가
        href_lists.append(href)
        
        first = driver.find_element_by_xpath('//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[2]').text
        print(first)
        regist_list.append("20" + first)
        
        ## 이미지
        thumb = driver.find_element_by_xpath('//*[@id="content"]/div[1]/button/div/img').get_attribute('src')
#         print(thumb)
        img_list.append(thumb)
        
        ## 저자
#         print(authors[i])
#         author_list.append(authors[idx])
        author_list.append(author_info)
    
        ## 요일
#         print(day)
        day_list.append(day)
    
        ## 줄거리
        story = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/p').text
#         print(story)
        story_list.append(story)
    
        ## 장르 & 태그
        tags = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/div/div[2]/div/div/a')
        temp = ""
        genre = tags[0].text.replace("#", "")
        genre_list.append(genre)
        
        for tag in tags:
#             print(tag.text)
            temp += tag.text.replace("#", "") + " "
        tag_list.append(temp)
        
        ## 관심 수
        viewer = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/button[1]/span[2]').text
#         print(viewer)
        viewer_list.append(viewer)
    
        ## 총 회차
        try:
            series = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[1]/div[1]').text
        except:
            series = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/div[1]').text
            #         print(series)
        series = series.replace("총 ", "")
        series = series.replace("화", "")
        series_list.append(series)
        
        platform_list.append("네이버")
    
        driver.back()
        
        idx += 1

In [ ]:
total_data = pd.DataFrame()
total_data['id'] = id_list
total_data['title'] = title_list
total_data['img'] = img_list ## 썸네일
total_data['author'] = author_list
total_data['day'] = day_list
total_data['genre'] = genre_list
total_data['story'] = story_list
total_data['tag'] = tag_list
total_data['viewer'] = viewer_list
total_data['grade'] = grade_list
total_data['regist'] = regist_list
total_data['series'] = series_list
total_data['href'] = href_lists
total_data['platform'] = platform_list
total_data.to_csv('new_href_네이버_웹툰2.csv', encoding = 'utf-8-sig')

In [ ]:
##### 완결 ######
driver.get('https://comic.naver.com/webtoon?tab=finish') #요일별로 링크 가져옴

html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')
    
sleep(0.5)

#스크롤 전 높이
before_h = driver.execute_script("return window.scrollY")

#무한 스크롤
while True:
    #맨 아래로 스크롤을 내린다
    driver.find_element_by_css_selector("body").send_keys(Keys.END)

    #스크롤 사이 페이지 로딩 시간
    sleep(1)

    #스크롤 후 높이
    after_h = driver.execute_script("return window.scrollY")

    if after_h == before_h:
        break

    before_h = after_h
        
## 웹툰 디테일 링크 + 별점

href_list = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/ul/li/a')
grades = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/ul/li/div/div/span')
#     authors_li = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/ul/li/div/div/a')
    
page_list = []
for page in href_list:
    page_list.append(page.get_attribute('href'))

scores = []
for grade in grades:
    scores.append(grade.text.replace("별점", "").replace("\n", ""))

#     authors = []
#     for author in authors_li:
#         authors.append(author.text)
        
#     print(len(authors))
#     for i in range(len(authors))
#         print(authors[i])
    
idx = 0

for href in page_list :
#     href = href_list[i].get_attribute('href')
#     print(href_list[i].get_attribute('href'))
#     print(grades[i].text.replace("별점", "").replace("\n", ""))
#     print(authors[i].text)

    #스크롤 전 높이
    before_h = driver.execute_script("return window.scrollY")

    #무한 스크롤
    while True:
    #맨 아래로 스크롤을 내린다
        driver.find_element_by_css_selector("body").send_keys(Keys.END)

    #스크롤 사이 페이지 로딩 시간
        sleep(1)

        #스크롤 후 높이
        after_h = driver.execute_script("return window.scrollY")

        if after_h == before_h:
            break

        before_h = after_h

    try:
        author = driver.find_element_by_xpath('//*[@id="content"]/div[1]/ul/li[' + str(idx+1) + ']/div/div[1]/a')
    except:
        try:
            author = driver.find_element_by_xpath('//*[@id="content"]/div[1]/ul/li['+ str(idx+1) + ']/div/a[2]')
        except:
            try:
                author = driver.find_element_by_xpath('//*[@id="content"]/div[1]/ul/li[' + str(idx+1) + ']/div/div[1]/a')
            except:
                print("#########지나갑니다")
                continue
                
    print(author.text)
    author_info = author.text
        
    sleep(2)

    driver.get(href + "&page=1&sort=ASC") # 디테일 이동
    
    html = requests.get(href).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
    soup = BeautifulSoup(html, 'html.parser')
    
    sleep(2)
    
    day = ko_week[i]
    
        ## 제목
    title = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/h2').text
    if (title in title_list):  # 요일 두 개 이상이면 요일만 추가함
                day_list[title_list.index(title)] += ', ' + day
                driver.back()
                continue
                
#         print(title)
    title_list.append(title)
    
    grade_list.append(scores[idx])
    
    id_list.append(num) ; num += 1  # id 리스트에 추가
        
    first = driver.find_element_by_xpath('//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[2]').text
    print(first)
    regist_list.append("20" + first)
    
    ## 이미지
    thumb = driver.find_element_by_xpath('//*[@id="content"]/div[1]/button/div/img').get_attribute('src')
#         print(thumb)
    img_list.append(thumb)
        
    ## 저자
#         print(authors[i])
#         author_list.append(authors[idx])
    author_list.append(author_info)
    
    ## 요일
#         print(day)
    day_list.append(day)
    
    ## 줄거리
    story = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/p').text
#         print(story)
    story_list.append(story)
    
    ## 장르 & 태그
    tags = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/div/div[2]/div/div/a')
    temp = ""
    genre = tags[0].text.replace("#", "")
    genre_list.append(genre)
        
    for tag in tags:
#             print(tag.text)
        temp += tag.text.replace("#", "") + " "
    tag_list.append(temp)
        
        ## 관심 수
    viewer = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/button[1]/span[2]').text
#         print(viewer)
    viewer_list.append(viewer)
    
    ## 총 회차
    try:
        series = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[1]/div[1]').text
    except:
        series = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/div[1]').text
            #         print(series)
    series = series.replace("총 ", "")
    series = series.replace("화", "")
    series_list.append(series)
        
    platform_list.append("네이버")
    
    driver.back()
        
    idx += 1